In [ ]:
from torch.utils.data import DataLoader
from utils.readers import VtkReader
from utils.patch_generator import GeneratePatches
import os
import numpy as np
import torch

In [ ]:
main_directory = os.path.dirname(os.getcwd())
general_path =  os.path.join(main_directory, 'patients_data', '{}', 'train')
images_path = general_path.format('MRI_volumes', 'MRI')
masks_path = general_path.format('masks', 'mask')
print(images_path)
print(masks_path)

In [ ]:
patient_id = 8
patient_image_path = os.path.join(images_path, 'MRI_{}.vtk'.format(patient_id))
patient_mask_path = os.path.join(masks_path, 'mask_{}.vtk'.format(patient_id))
print(patient_image_path)
print(patient_mask_path)

In [ ]:
image = VtkReader(patient_image_path)
mask = VtkReader(patient_mask_path)
print(image.shape)
print(image.shape)
print(np.sum(image==1))

In [ ]:
print(np.sum(mask))
print(np.prod(mask.shape))
print(np.sum(mask)/np.prod(mask.shape))

In [ ]:
ddd = GeneratePatches(images_path, 
                      masks_path, 
                      (16,16, 6), 
                      shuffle_images=True, 
                      allow_overlap=True)

In [ ]:
for p in ddd.patients_information:
    print(p)

In [ ]:
ddd[4000][0].shape